In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=fc5b7bc487c861aa71916a18fd082b168fba631e2d41ea8dd624f3397874eb28
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-02 20:07:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.30MB/s    in 0.2s    

2023-04-02 20:07:35 (6.30 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Import SparkFiles and create Dataframe of Musical instrument reviews
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...|2015-08-31 00:00:00|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser

In [5]:
# Filter DataFrames for votes greater than/equal to 20
review_df = df.filter(df["total_votes"]>= 20)
review_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   15365576|R2243Y3OD8U6KQ|B00W1RTVGO|     160618734|Supertech stage l...|Musical Instruments|          5|           47|         61|   N|                N|This fills a room...|I am always looki...|2015-08-31 00:00:00|
|         US|   28770559|R2TGT0CDTCAAHW|B00INJ7HBK|     157027184|Singing Ma

In [6]:
# Filter the Review DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes = review_df.filter(review_df["helpful_votes"]/review_df["total_votes"]>= 0.5)
helpful_votes.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   15365576|R2243Y3OD8U6KQ|B00W1RTVGO|     160618734|Supertech stage l...|Musical Instruments|          5|           47|         61|   N|                N|This fills a room...|I am always looki...|2015-08-31 00:00:00|
|         US|   28770559|R2TGT0CDTCAAHW|B00INJ7HBK|     157027184|Singing Ma

In [7]:
# Create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program
paid_df = helpful_votes.filter(helpful_votes["vine"]=="Y")
paid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42689422|R1R9RU7JW0MFR2|B0124Y99PQ|     618027384|Casio CGP-700BK C...|Musical Instruments|          4|           20|         23|   Y|                N|Be prepared to be...|First off PLEASE ...|2015-08-27 00:00:00|
|         US|   29182364|R19EFYNN3W8Q07|B00ZU4G0ZK|     499223759|TC Electro

In [9]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_df = helpful_votes.filter(helpful_votes["vine"]=="N")
unpaid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   15365576|R2243Y3OD8U6KQ|B00W1RTVGO|     160618734|Supertech stage l...|Musical Instruments|          5|           47|         61|   N|                N|This fills a room...|I am always looki...|2015-08-31 00:00:00|
|         US|   28770559|R2TGT0CDTCAAHW|B00INJ7HBK|     157027184|Singing Ma

In [10]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for PAID reviews.
paid_df.describe().show()

+-------+-----------+--------------------+--------------+----------+--------------------+--------------------+-------------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+
|summary|marketplace|         customer_id|     review_id|product_id|      product_parent|       product_title|   product_category|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|     review_headline|         review_body|
+-------+-----------+--------------------+--------------+----------+--------------------+--------------------+-------------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+
|  count|         60|                  60|            60|        60|                  60|                  60|                 60|                60|                60|               60|  60|               60|                  60|              

In [11]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for UNPAID reviews.
unpaid_df.describe().show()

+-------+-----------+-------------------+-------------+--------------------+-------------------+--------------------+-------------------+------------------+-----------------+-----------------+-----+-----------------+--------------------+--------------------+
|summary|marketplace|        customer_id|    review_id|          product_id|     product_parent|       product_title|   product_category|       star_rating|    helpful_votes|      total_votes| vine|verified_purchase|     review_headline|         review_body|
+-------+-----------+-------------------+-------------+--------------------+-------------------+--------------------+-------------------+------------------+-----------------+-----------------+-----+-----------------+--------------------+--------------------+
|  count|      14477|              14477|        14477|               14477|              14477|               14477|              14477|             14477|            14477|            14477|14477|            14477|       

In [14]:
# Calculate total number of paid reviews.
paid_number = paid_df.count()
print(paid_number)

60


In [15]:
# Calculate total number of paid 5-star reviews.
paid_fivestar_number = paid_df[paid_df["star_rating"]==5].count()
print(paid_fivestar_number)

34


In [16]:
# Calculate the percentage of 5-star reviews that are paid.
paid_fivestar_percentage = paid_fivestar_number/paid_number
print(paid_fivestar_percentage)

0.5666666666666667


In [17]:
# Calculate total number of unpaid reviews.
unpaid_number = unpaid_df.count()
print(unpaid_number)

14477


In [18]:
# Calculate total number of unpaid 5-star reviews.
unpaid_fivestar_number = unpaid_df[paid_df["star_rating"]==5].count()
print(unpaid_fivestar_number)

8212


In [19]:
# Calculate the percentage of 5-star reviews that are unpaid.
unpaid_fivestar_percentage = unpaid_fivestar_number/unpaid_number
print(unpaid_fivestar_percentage)

0.5672445948746287
